In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import seaborn as sns

In [20]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Load dataset
df = pd.read_csv("manufacturing_dataset_1000_samples.csv")

# Keep only required features + target
required_cols = [
    "Injection_Temperature", "Injection_Pressure", "Cycle_Time",
    "Cooling_Time", "Material_Viscosity", "Ambient_Temperature",
    "Machine_Age", "Machine_Type", "Day_of_Week"   # <-- target column
]
df = df[required_cols]

# Define target and features
y = df["Day_of_Week"]
X = df.drop(columns=["Day_of_Week"])

# Separate categorical & numerical
categorical_cols = ["Machine_Type"]
numeric_cols = [col for col in X.columns if col not in categorical_cols]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

# Pipeline with Random Forest
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf.fit(X_train, y_train)

# Save model
with open("model.pkl1", "wb") as f:
    pickle.dump(clf, f)

print("✅ Model trained only with selected columns and saved!")


✅ Model trained only with selected columns and saved!


In [5]:
df.head()

,Timestamp,Injection_Temperature,Injection_Pressure,Cycle_Time,Cooling_Time,Material_Viscosity,Ambient_Temperature,Machine_Age,Operator_Experience,Maintenance_Hours,Shift,Machine_Type,Material_Grade,Day_of_Week,Temperature_Pressure_Ratio,Total_Cycle_Time,Efficiency_Score,Machine_Utilization,Parts_Per_Hour
0,2023-01-01 00:00:00,221.0,136.0,28.7,13.6,375.5,28.0,3.8,11.2,64,Evening,Type_B,Economy,Thursday,1.625,42.3,0.063,0.510,36.5
1,2023-01-01 01:00:00,213.3,128.9,34.5,14.0,215.8,22.6,6.8,6.3,58,Night,Type_A,Standard,Wednesday,1.655,48.5,0.037,0.389,29.9
2,2023-01-01 02:00:00,222.8,115.9,19.9,9.5,307.0,25.3,4.2,9.6,47,Day,Type_A,Standard,Monday,1.922,29.4,0.061,0.551,56.9
3,2023-01-01 03:00:00,233.3,105.3,39.2,13.1,137.8,26.0,9.2,8.6,49,Evening,Type_A,Premium,Saturday,2.215,52.3,0.054,0.293,31.0
4,2023-01-01 04:00:00,212.2,125.5,45.0,9.9,298.2,23.6,6.2,23.0,49,Night,Type_B,Premium,Monday,1.691,54.9,0.145,0.443,15.0


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load dataset
df = pd.read_csv("manufacturing_dataset_1000_samples.csv")

# Drop unwanted columns
drop_cols = ['Machine_Utilization', 'Parts_Per_Hour', 'Total_Cycle_Time',
             'Efficiency_Score', 'Temperature_Pressure_Ratio']
df = df.drop(columns=drop_cols, errors='ignore')

# Define features and target
X = df.drop(columns=['Shift'])   # Target column
y = df['Shift']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline
pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=3)),
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=42))
])

# Train
pipeline.fit(X_train, y_train)

# Save model
with open("model.pkl", "wb") as f:
    pickle.dump((pipeline, X.columns), f)  # Save also the column order


ValueError: could not convert string to float: '2023-01-02 05:00:00'